# Coxswain Rotation

The code here figures out whose turn is to cox for the LMRC Mixed Masters team. It uses the TeamSnap API to retrieve the list of members available for an event, compares this list against the list of dates when each member last coxed, and then orders the available members by the date they last coxed.

The following Python packages are required to run the full notebook:

`yaml
requests
pandas
Levenshtein
tabulate`

The code was only tested in Python 3...

In [ ]:
# This should be the only part of the code that needs to be changed.
# The event ID is listed in the address bar when opening the event page 
# on TeamSnap. If there is an easier way to get the ID of the next event, 
# I have not figured it out yet...

EVENT_ID = 15880077

## Retrieve List of Available Team Members

Here is a summary of how to generate the required credentials and access the API:

http://developer.teamsnap.com/documentation/apiv3/authorization/#token-authentication-flow

### Generate Authentication URL for the TeamSnap API

In [ ]:
import yaml

with open("credentials/cred.yml") as f:
    cred = yaml.load(f)
client_id = cred['client_id']
secret = cred['secret']
uri = cred['callback_uri']

RESPONSE_TYPE = 'token'
AUTHORIZATION_URL = "https://auth.teamsnap.com/oauth/authorize"

# Build URL
authentication_url = AUTHORIZATION_URL + \
                     "?client_id=" + client_id + \
                     "&redirect_uri=" + uri + \
                     "&response_type=" + RESPONSE_TYPE + \
                     "&scope=read"

In [ ]:
print("""Visit the URL below. You will be redirected to a web page 
containing basic information about the application. In the HTML
address of the page, you will see a long token listed after 
'access_token='. Copy this token and use it for the next step.\n""")
print(authentication_url)

### List Available Members

The token obtained in the previous step is read from the YAML file `credentials/cred.yml`. Just like the other credentials, the token should not be made public, so do not include it in this code. Modify the YAML file as necessary. The token is used to retrieve data via the API.

The list of available members only associates availability with the member ID, rather than with the member name. To connect the name with the availability status, I first retrieve the list of team members (connects name with member ID) and only then the list of availabilities (connects member ID with availability status).

In [ ]:
import requests

oauth_token = cred['oauth_token']

#### Get Member List

In [ ]:
# Read the team ID from the YAML file. 
TEAM_ID = cred['team_id']

# Retrieve the member list as JSON.
url = 'https://api.teamsnap.com/v3/members/search?team_id=' + str(TEAM_ID)
headers = {'Content-Type': 'application/json', 
           'Accept-Charset': 'UTF-8', 
           'Authorization': 'Bearer ' + oauth_token}
json_req = requests.get(url, headers=headers).json()

In [ ]:
# Add the members to a dictionary. The dictionary will 
# be indexed by member ID, and each (unique) member ID 
# is associated with a member name. 

member_dict = {}
n_team_members = len(json_req['collection']['items'])
for i_user in range(n_team_members):
    member_id = json_req['collection']['items'][i_user]['data'][0]['value']
    name = json_req['collection']['items'][i_user]['data'][9]['value'] + ' ' + \
           json_req['collection']['items'][i_user]['data'][32]['value']
    # Fix issues with multiple/trailing spaces in some TeamSnap entries.
    member_dict[member_id] = ' '.join(name.split())

#### Get List of Available Members

In [ ]:
# Retrieve the list of available members as JSON.
url = 'https://api.teamsnap.com/v3/availabilities/search?team_id=' + str(TEAM_ID) + \
      '&event_id=' + str(EVENT_ID)
headers = {'Content-Type': 'application/json', 
           'Accept-Charset': 'UTF-8', 
           'Authorization': 'Bearer ' + oauth_token}
json_req = requests.get(url, headers=headers).json()

In [ ]:
# Connect the list of available members with the member names, 
# and add the available names to a dictionary. The dictionary 
# is indexed by availability status, and only members that 
# replied with 'Yes' or 'Maybe' are included.

availability_dict = {'available': [], 'maybe': []}
for i_user in range(n_team_members):
    member_id = json_req['collection']['items'][i_user]['data'][3]['value']
    availability = json_req['collection']['items'][i_user]['data'][6]['value']
    if availability == "Yes. I will be there.":
        availability_dict['available'].append(member_dict[member_id])
    elif availability == "Maybe. I'm not sure yet.":
        availability_dict['maybe'].append(member_dict[member_id])

Print the list of members coming to practice.

In [ ]:
available_members = sorted(availability_dict['available'])
maybes_members = sorted(availability_dict['maybe'])

n_available = len(available_members)
print("\n".join(available_members))
print("\n" + str(n_available) + " team members available for practice.\n\n")

print("\n".join(maybes_members))
n_maybes = len(maybes_members)
print("\n" + str(n_maybes) + " team members might also come for practice.\n\n")

## Who's Next?

In [ ]:
from datetime import datetime as dt
import pandas as pd
from Levenshtein import jaro

### Get List of Last Coxing Dates

In [ ]:
coxswain_data = pd.read_csv("coxswain_rotation.csv", header=0, index_col='Name')
coxswain_data['Date'] = pd.to_datetime(coxswain_data['Date'], format='%m/%d/%y')

# Some names might be badly formatted, so things like trailing spaces, 
# nicknames, and typos should be checked for.

# Remove trailing spaces.
coxswain_data.index = coxswain_data.index.str.strip().str.replace('  ', ' ')

### Get Most Recent Coxing Dates of Available Members

In [ ]:
def get_cox_dates(members, coxswain_data):
    '''Return most recent dates when team members coxed.'''
    cox_dates = {}
    for name in members: 
        if name in coxswain_data.index:
            date = coxswain_data.loc[name]['Date']
            # Some names appear twice in the coxswain rotation document, 
            # in which case the most recent date should be used.
            if isinstance(date, pd.core.series.Series):
                date = date.max()
            cox_dates[name] = date
        else:
            # If Jaro distance is above a certain value, then the names 
            # are very similar and probably different only because of a typo 
            # or nickname.
            # TODO: Is 0.8 a good value? Works for Mo's name... :)
            jaro_dist_thresh = 0.8
            found_approx_name = False
            for cox_name in coxswain_data.index:
                if jaro(cox_name, name) > jaro_dist_thresh:
                    found_approx_name = True
                    date = coxswain_data.loc[cox_name]['Date']
                    # Some of the mispelled names might appear twice too, in which 
                    # case the latest date should be considered.
                    if name in cox_dates and date < cox_dates[name]:
                        continue
                    else:
                        cox_dates[name] = date
            if not found_approx_name:
                cox_dates[name] = dt.strptime('1/1/1900', '%m/%d/%Y')
    return cox_dates

cox_dates_available = get_cox_dates(availability_dict['available'], coxswain_data)
cox_dates_maybes = get_cox_dates(availability_dict['maybe'], coxswain_data)

In [ ]:
import operator

# Sort dictionary by the date when people last coxed.
# Returns a list of tuples of the form (member_name, last_coxing_date).
next_coxswains_available = sorted(cox_dates_available.items(), key=operator.itemgetter(1))
next_coxswains_maybe = sorted(list(cox_dates_maybes.items()) + list(cox_dates_available.items()), 
                              key=operator.itemgetter(1))

Print the names of the members coming to practice, in reverse order of their most recent coxing date.

In [ ]:
from tabulate import tabulate

def coxswains(c_list):
    never_coxed, coxed_before = [], []
    for c in c_list:
        if c[1].year == 1900:
            never_coxed.append(c[0])
        else:
            coxed_before.append((c[0], c[1].strftime("%m/%d/%Y")))
    return coxed_before, never_coxed

def print_coxes(coxed_before, never_coxed):
    print('People Who Coxed Before: \n')
    print(tabulate(coxed_before, headers=['Name', 'Date'], tablefmt='orgtbl'))

    print('\n')
    print('People Who Never Coxed: \n')
    for c in never_coxed:
        print(c)

coxed_before, never_coxed = coxswains(next_coxswains_available)
print('AVAILABLE MEMBERS ONLY:\n')    
print_coxes(coxed_before, never_coxed)

coxed_before, never_coxed = coxswains(next_coxswains_maybe)
print("\n\nAVAILABLE & 'MAYBE' MEMBERS:\n")
print_coxes(coxed_before, never_coxed)